In [1]:
%matplotlib inline
import numpy as np
%load_ext autoreload
%autoreload 2

from implementation import *

# Data

In [2]:
y,x,ids=load_csv_data("train.csv") 

x[x==-999]=np.NaN
z=x[:,~np.isnan(x).any(axis=0)]

# Building k indices

In [3]:
def build_k_indices(y, K, seed):
    """build k array of indices of y"""
    num_row = y.shape[0]
    interval = int(num_row / K)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    K_indices = [indices[k * interval: (k + 1) * interval] for k in range(K)]
    return np.array(K_indices)

In [4]:
# ça nous donne un array de K array de y.shape[0]/K INDICES
print(build_k_indices(y, 5, 1))

[[240208  18744 207175 ..., 146611 152140 168265]
 [130575 246655  84533 ..., 246477  41107  33329]
 [234919 162743 198590 ..., 176431  43557 189081]
 [ 96064 224056 233462 ..., 113640 108081  99162]
 [146937  96870  25428 ..., 208780 229611 128037]]


# Cross-Validation

In [5]:
def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression"""
    # set k is test set, the others are the training set
    te_indice = k_indices[k]
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    y_te = y[te_indice]
    y_tr = y[tr_indice]
    x_te = x[te_indice]
    x_tr = x[tr_indice]
    
    # form data with polynomial degree, using build_poly
    tx_tr = build_poly(x_tr, degree)
    tx_te = build_poly(x_te, degree)
    
    # perform ridge regression
    w = ridge_regression(y_tr, tx_tr, lambda_)
    
    # calculate the loss for train and test data   A NOTER: ici, on calcule sqrt(2*mse) POURQUOI ?
    loss_tr = np.sqrt(2 * compute_mse(y_tr, tx_tr, w))
    loss_te = np.sqrt(2 * compute_mse(y_te, tx_te, w))
    return loss_tr, loss_te,w

In [19]:
""" Remarquez le résultat loufoque pour le 3ème, erreur énorme pour test"""
for i in range(4):
    print(cross_validation(y,z,build_k_indices(y,4,1),i,0.7,3))
    loss1,loss2,w = cross_validation(y,z,build_k_indices(y,4,1),i,0.7,3)
    if (i==0):
        w_final, loss_final = w, loss2
    else:
        if (loss_final > loss2):
            w_final, loss_final = w, loss2
print(loss_final, w_final)

(0.8272602530356844, 0.82791807574905574, array([ -8.24848890e-03,  -1.57055919e-02,   4.01312724e-03,
        -4.16549753e-04,   3.73844493e-03,  -2.90781669e-03,
         2.16366109e-03,  -1.00555109e-02,   1.46221873e-02,
         1.30995830e-02,  -7.44309842e-04,  -1.11963093e-03,
        -1.12563795e-02,  -8.37414703e-05,   5.99799304e-04,
        -7.96165778e-04,   2.46189805e-04,  -9.32262396e-04,
        -1.59653335e-03,   3.20066298e-04,   7.22026500e-05,
        -2.65704469e-05,   2.90479109e-06,   1.75637163e-02,
         1.96757232e-05,   1.68431734e-06,  -6.69216306e-03,
         1.21894203e-02,  -7.06652286e-05,  -2.39769415e-02,
        -5.72596131e-03,   9.65015991e-05,  -3.59079008e-02,
        -5.81695683e-03,   1.44244038e-05,  -3.81915783e-03,
         1.25559267e-06,   3.03099868e-03,  -9.85612600e-06,
        -8.33215532e-08,   2.08524641e-08,  -5.89372238e-09,
         4.77853871e-04,   4.70866626e-09,   1.39489921e-10,
         7.22751509e-04,   2.75947897e-02, 